<a href="https://colab.research.google.com/github/Vishnupriya-Selvraj/Agentic_AI_Workshop/blob/main/RAG_System_AI_Research_Papers_Q%26A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Cell 1: Installation and Setup**

**Installs core dependencies:**

In [87]:
!pip install langchain langchain-community langchain-huggingface
!pip install sentence-transformers faiss-cpu
!pip install pypdf2 PyPDF2
!pip install transformers torch
!pip install gradio
!pip install accelerate

print("✅ All packages installed successfully!")

✅ All packages installed successfully!


**Key Packages:**

*  langchain - Framework for building RAG
systems

*   sentence-transformers - For generating text embeddings

*   faiss-cpu - Efficient vector similarity search

*   PyPDF2 - PDF text extraction

*   transformers - Language models for answer generation

*   gradio - Web interface builder

# **Cell 2: Import Libraries**
**Loads all required modules:**

In [88]:
import os
import warnings
warnings.filterwarnings('ignore')

import torch
from typing import List, Dict, Any
import gradio as gr
from pathlib import Path

# LangChain imports
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document

# Transformers
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

print("✅ All libraries imported successfully!")
print(f"🔥 GPU Available: {torch.cuda.is_available()}")

✅ All libraries imported successfully!
🔥 GPU Available: False


**Critical Components:**

*   PDF processing tools (PyPDFLoader)

*   Embedding models (HuggingFaceEmbeddings)

*   Vector database (FAISS)

*   GPU check (torch.cuda.is_available())

# **Cell 3: Document Processor**
**Handles PDF loading and preprocessing:**

In [89]:
class DocumentProcessor:
    """Handles loading and processing of PDF documents"""

    def __init__(self, chunk_size=1000, chunk_overlap=200):
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            length_function=len,
            separators=["\n\n", "\n", ".", " ", ""]
        )

    def load_pdfs(self, pdf_paths: List[str]) -> List[Document]:
        """Load multiple PDF files and return documents"""
        all_documents = []

        for pdf_path in pdf_paths:
            try:
                print(f"📖 Loading: {os.path.basename(pdf_path)}")
                loader = PyPDFLoader(pdf_path)
                documents = loader.load()

                # Add metadata
                for i, doc in enumerate(documents):
                    doc.metadata.update({
                        'source_file': os.path.basename(pdf_path),
                        'page_number': i + 1,
                        'total_pages': len(documents)
                    })

                all_documents.extend(documents)
                print(f"✅ Loaded {len(documents)} pages from {os.path.basename(pdf_path)}")

            except Exception as e:
                print(f"❌ Error loading {pdf_path}: {str(e)}")

        return all_documents

    def create_chunks(self, documents: List[Document]) -> List[Document]:
        """Split documents into chunks for better retrieval"""
        print("🔪 Creating text chunks...")

        chunks = self.text_splitter.split_documents(documents)

        # Add chunk metadata
        for i, chunk in enumerate(chunks):
            chunk.metadata.update({
                'chunk_id': i,
                'chunk_size': len(chunk.page_content),
                'processing_timestamp': str(pd.Timestamp.now())
            })

        print(f"✅ Created {len(chunks)} chunks")
        return chunks

# Test the document processor
print("\n🧪 Testing Document Processor...")
doc_processor = DocumentProcessor()
print("Document processor ready!")


🧪 Testing Document Processor...
Document processor ready!


**Key Features:**

*   Chunk size customization (default: 1000 chars)

*   Metadata preservation (source file, page numbers)

*   Smart text splitting at natural boundaries

# **Cell 4: Vector Store Manager**
**Manages document embeddings:**

In [90]:
class VectorStoreManager:
    """Manages document embeddings and similarity search"""

    def __init__(self, model_name="sentence-transformers/all-MiniLM-L6-v2"):
        self.model_name = model_name
        self.embeddings = HuggingFaceEmbeddings(
            model_name=model_name,
            model_kwargs={'device': 'cuda' if torch.cuda.is_available() else 'cpu'},
            encode_kwargs={'normalize_embeddings': True}
        )
        self.vector_store = None

    def create_vector_store(self, chunks: List[Document]) -> bool:
        """Create FAISS vector store from document chunks"""
        try:
            print("⚡ Creating vector embeddings...")
            print(f"📊 Processing {len(chunks)} chunks...")

            # Create embeddings and vector store
            self.vector_store = FAISS.from_documents(
                documents=chunks,
                embedding=self.embeddings
            )

            print(f"✅ Vector store created successfully!")
            print(f"📈 Vector dimension: {self.vector_store.index.d}")
            print(f"📚 Total vectors: {self.vector_store.index.ntotal}")

            return True

        except Exception as e:
            print(f"❌ Error creating vector store: {str(e)}")
            return False

    def search_similar(self, query: str, k: int = 5) -> List[tuple]:
        """Search for similar documents"""
        if not self.vector_store:
            return []

        try:
            # Get documents with similarity scores
            results = self.vector_store.similarity_search_with_score(query, k=k)

            print(f"🔍 Found {len(results)} relevant chunks for query")

            return results

        except Exception as e:
            print(f"❌ Search error: {str(e)}")
            return []

    def get_stats(self) -> Dict[str, Any]:
        """Get vector store statistics"""
        if not self.vector_store:
            return {"status": "not_created"}

        return {
            "status": "ready",
            "total_vectors": self.vector_store.index.ntotal,
            "vector_dimension": self.vector_store.index.d,
            "model_name": self.model_name
        }

# Test vector store manager
print("\n🧪 Testing Vector Store Manager...")
vector_manager = VectorStoreManager()
print("Vector store manager ready!")


🧪 Testing Vector Store Manager...
Vector store manager ready!


**Technical Details:**

*   Uses **`all-MiniLM-L6-v2`** embedding model

*   Automatic GPU/CPU switching

*   Returns similarity scores with results

# **Cell 5: Answer Generator**
**Produces human-readable answers:**

In [91]:
class AnswerGenerator:
    """Generates answers using retrieved context"""

    def __init__(self):
        self.generator = None
        self.model_name = "microsoft/DialoGPT-medium"
        self.setup_model()

    def setup_model(self):
        """Initialize the text generation model"""
        try:
            print("🤖 Loading language model...")

            # Use a simple but effective model for answer generation
            self.generator = pipeline(
                "text-generation",
                model=self.model_name,
                tokenizer=self.model_name,
                max_length=512,
                temperature=0.7,
                do_sample=True,
                pad_token_id=50256,
                device=0 if torch.cuda.is_available() else -1
            )

            print("✅ Language model loaded!")

        except Exception as e:
            print(f"❌ Error loading model: {str(e)}")
            # Fallback to a simpler approach
            self.generator = None

    def generate_answer(self, question: str, context: str, max_length: int = 200) -> str:
        """Generate answer based on context"""

        if not self.generator:
            return self._simple_answer(question, context)

        try:
            # Create a focused prompt
            prompt = f"""
Based on the research paper context below, answer the question concisely and accurately.

Context: {context[:1000]}...

Question: {question}

Answer:"""

            # Generate response
            response = self.generator(
                prompt,
                max_length=len(prompt) + max_length,
                num_return_sequences=1,
                temperature=0.3,
                do_sample=True
            )

            # Extract just the answer part
            full_response = response[0]['generated_text']
            answer = full_response[len(prompt):].strip()

            # Clean up the answer
            answer = self._clean_answer(answer)

            return answer if answer else self._simple_answer(question, context)

        except Exception as e:
            print(f"⚠️ Generation error: {str(e)}")
            return self._simple_answer(question, context)

    def _simple_answer(self, question: str, context: str) -> str:
        """Fallback method for answer generation"""
        # Simple extractive approach
        sentences = context.split('.')
        relevant_sentences = []

        question_words = set(question.lower().split())

        for sentence in sentences[:5]:  # Take first 5 sentences
            sentence_words = set(sentence.lower().split())
            if question_words.intersection(sentence_words):
                relevant_sentences.append(sentence.strip())

        if relevant_sentences:
            return ". ".join(relevant_sentences[:2]) + "."
        else:
            return "Based on the provided context, " + sentences[0][:200] + "..."

    def _clean_answer(self, answer: str) -> str:
        """Clean and format the generated answer"""
        # Remove common artifacts
        answer = answer.replace('\n', ' ')
        answer = ' '.join(answer.split())  # Remove extra spaces

        # Ensure it ends properly
        if answer and not answer.endswith(('.', '!', '?')):
            answer += '.'

        return answer

# Test answer generator
print("\n🧪 Testing Answer Generator...")
answer_gen = AnswerGenerator()
print("Answer generator ready!")



🧪 Testing Answer Generator...
🤖 Loading language model...


config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/863M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/863M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

Device set to use cpu


✅ Language model loaded!
Answer generator ready!


**Generation Pipeline:**

*   Creates LLM prompt with question + context

*   Uses DialoGPT-medium for generation

*   Fallback to extractive QA if generation fails

# **Cell 6: RAG System**
**Main orchestrator class:**

In [92]:
class RAGSystem:
    """Complete Retrieval-Augmented Generation system"""

    def __init__(self):
        self.doc_processor = DocumentProcessor()
        self.vector_manager = VectorStoreManager()
        self.answer_generator = AnswerGenerator()
        self.is_ready = False
        self.documents_info = {}

    def setup(self, pdf_paths: List[str]) -> bool:
        """Complete system setup"""
        print("🚀 Setting up RAG system...")
        print("="*50)

        try:
            # Step 1: Load documents
            print("Step 1: Loading PDFs...")
            documents = self.doc_processor.load_pdfs(pdf_paths)

            if not documents:
                print("❌ No documents loaded!")
                return False

            # Step 2: Create chunks
            print("\nStep 2: Creating text chunks...")
            chunks = self.doc_processor.create_chunks(documents)

            # Step 3: Create vector store
            print("\nStep 3: Creating vector embeddings...")
            if not self.vector_manager.create_vector_store(chunks):
                return False

            # Step 4: Store document information
            self.documents_info = {
                'total_documents': len(documents),
                'total_chunks': len(chunks),
                'pdf_files': [os.path.basename(path) for path in pdf_paths],
                'setup_complete': True
            }

            self.is_ready = True
            print("\n" + "="*50)
            print("🎉 RAG System Setup Complete!")
            print(f"📚 Loaded: {len(documents)} pages from {len(pdf_paths)} PDFs")
            print(f"🔪 Created: {len(chunks)} text chunks")
            print(f"⚡ Vector store ready with {self.vector_manager.vector_store.index.ntotal} embeddings")
            print("="*50)

            return True

        except Exception as e:
            print(f"❌ Setup failed: {str(e)}")
            return False

    def ask_question(self, question: str, num_sources: int = 3) -> Dict[str, Any]:
        """Ask a question and get an answer with sources"""

        if not self.is_ready:
            return {
                'answer': "❌ System not ready. Please upload and process documents first.",
                'sources': [],
                'confidence': 0.0
            }

        if not question.strip():
            return {
                'answer': "❌ Please provide a valid question.",
                'sources': [],
                'confidence': 0.0
            }

        try:
            print(f"🔍 Processing question: '{question}'")

            # Step 1: Retrieve relevant documents
            search_results = self.vector_manager.search_similar(question, k=num_sources)

            if not search_results:
                return {
                    'answer': "❌ No relevant information found in the documents.",
                    'sources': [],
                    'confidence': 0.0
                }

            # Step 2: Prepare context and source information
            context_parts = []
            sources = []
            total_score = 0

            for doc, score in search_results:
                context_parts.append(doc.page_content)
                total_score += (1 - score)  # Convert distance to similarity

                source_info = {
                    'file': doc.metadata.get('source_file', 'Unknown'),
                    'page': doc.metadata.get('page_number', 'Unknown'),
                    'chunk_id': doc.metadata.get('chunk_id', 'Unknown'),
                    'similarity_score': round(1 - score, 3),  # Convert to similarity
                    'content_preview': doc.page_content[:150] + "..." if len(doc.page_content) > 150 else doc.page_content
                }
                sources.append(source_info)

            # Step 3: Combine context
            combined_context = "\n\n".join(context_parts)

            # Step 4: Generate answer
            print("🤖 Generating answer...")
            answer = self.answer_generator.generate_answer(question, combined_context)

            # Step 5: Calculate confidence score
            confidence = min(total_score / len(search_results), 1.0)

            result = {
                'answer': answer,
                'sources': sources,
                'confidence': round(confidence, 3),
                'context_length': len(combined_context),
                'num_sources_used': len(sources)
            }

            print(f"✅ Answer generated (confidence: {result['confidence']})")
            return result

        except Exception as e:
            print(f"❌ Error processing question: {str(e)}")
            return {
                'answer': f"❌ Error processing your question: {str(e)}",
                'sources': [],
                'confidence': 0.0
            }

    def get_system_status(self) -> Dict[str, Any]:
        """Get current system status and statistics"""
        base_status = {
            'is_ready': self.is_ready,
            'documents_info': self.documents_info,
            'vector_store_stats': self.vector_manager.get_stats()
        }

        if self.is_ready:
            base_status['model_info'] = {
                'embedding_model': self.vector_manager.model_name,
                'generation_model': self.answer_generator.model_name,
                'gpu_available': torch.cuda.is_available()
            }

        return base_status

# Initialize the complete RAG system
print("\n🧪 Initializing Complete RAG System...")
rag_system = RAGSystem()
print("✅ RAG system ready for setup!")



🧪 Initializing Complete RAG System...
🤖 Loading language model...


Device set to use cpu


✅ Language model loaded!
✅ RAG system ready for setup!


**Workflow:**
PDFs → Chunks → Embeddings → Vector Store → Query → Answer

# **Cell 7: File Helpers**
**Colab-specific utilities:**

In [93]:
import pandas as pd
from google.colab import files
import shutil

def upload_pdf_files():
    """Helper function to upload PDF files in Colab"""
    print("📤 Please upload your PDF files...")
    uploaded = files.upload()

    pdf_paths = []
    for filename, data in uploaded.items():
        if filename.endswith('.pdf'):
            # Save file to current directory
            with open(filename, 'wb') as f:
                f.write(data)
            pdf_paths.append(filename)
            print(f"✅ Saved: {filename}")
        else:
            print(f"⚠️ Skipped non-PDF file: {filename}")

    return pdf_paths

def setup_with_sample_data():
    """Setup system with sample data for demonstration"""
    print("🎯 Setting up with sample data...")
    print("Note: In real usage, upload your own PDF files using upload_pdf_files()")

    # Create a sample document for demonstration
    sample_content = """
    This is a sample AI research paper about Natural Language Processing.

    Abstract: This paper presents a novel approach to question answering systems
    using retrieval-augmented generation. Our method combines semantic search
    with large language models to provide accurate and contextual answers.

    Introduction: Question answering has been a fundamental challenge in NLP.
    Recent advances in transformer models have shown promising results.

    Methodology: We propose a hybrid approach that uses vector embeddings
    for document retrieval and generative models for answer synthesis.

    Results: Our system achieves 85% accuracy on benchmark datasets,
    outperforming traditional keyword-based approaches by 20%.

    Conclusion: The combination of retrieval and generation provides
    superior performance for domain-specific question answering tasks.
    """

    # Create sample PDF content (simplified)
    from langchain.schema import Document

    sample_doc = Document(
        page_content=sample_content,
        metadata={
            'source_file': 'sample_paper.pdf',
            'page_number': 1,
            'total_pages': 1
        }
    )

    # Process sample document
    chunks = rag_system.doc_processor.create_chunks([sample_doc])

    # Create vector store
    if rag_system.vector_manager.create_vector_store(chunks):
        rag_system.is_ready = True
        rag_system.documents_info = {
            'total_documents': 1,
            'total_chunks': len(chunks),
            'pdf_files': ['sample_paper.pdf'],
            'setup_complete': True
        }
        print("✅ Sample system ready for testing!")
        return True

    return False


**Usage Tips:**

*   Drag-and-drop PDF upload support

*   Sample data for quick testing

# **Cell 8: Gradio UI**
**Builds interactive interface:**

In [94]:
def create_gradio_interface():
    """Create an interactive web interface"""

    def handle_file_upload(files):
        if not files:
            return "❌ Please upload at least one PDF file."

        try:
            pdf_paths = []
            for file in files:
                if file.name.endswith('.pdf'):
                    pdf_paths.append(file.name)

            if not pdf_paths:
                return "❌ No valid PDF files found."

            # Setup the RAG system
            success = rag_system.setup(pdf_paths)

            if success:
                status = rag_system.get_system_status()
                return f"""✅ Setup Complete!

📊 System Status:
• Documents loaded: {status['documents_info']['total_documents']}
• Text chunks created: {status['documents_info']['total_chunks']}
• Files processed: {', '.join(status['documents_info']['pdf_files'])}
• Vector embeddings: {status['vector_store_stats']['total_vectors']}

🎯 Ready to answer questions!"""
            else:
                return "❌ Setup failed. Please check your PDF files and try again."

        except Exception as e:
            return f"❌ Error during setup: {str(e)}"

    def handle_question(question):
        if not question.strip():
            return "❌ Please enter a question.", ""

        result = rag_system.ask_question(question)

        # Format answer
        answer_text = f"**Answer:** {result['answer']}\n\n"
        answer_text += f"**Confidence:** {result['confidence']}/1.0\n\n"

        # Format sources
        sources_text = "**Sources:**\n\n"
        for i, source in enumerate(result['sources'], 1):
            sources_text += f"**{i}. {source['file']}** (Page {source['page']})\n"
            sources_text += f"   • Similarity: {source['similarity_score']}\n"
            sources_text += f"   • Preview: {source['content_preview']}\n\n"

        if not result['sources']:
            sources_text = "No sources found."

        return answer_text, sources_text

    def show_system_info():
        status = rag_system.get_system_status()

        if not status['is_ready']:
            return "System not ready. Please upload documents first."

        info_text = f"""**System Information:**

**Status:** {'✅ Ready' if status['is_ready'] else '❌ Not Ready'}

**Documents:**
• Total documents: {status['documents_info']['total_documents']}
• Total chunks: {status['documents_info']['total_chunks']}
• Files: {', '.join(status['documents_info']['pdf_files'])}

**Vector Store:**
• Total vectors: {status['vector_store_stats']['total_vectors']}
• Vector dimension: {status['vector_store_stats']['vector_dimension']}
• Embedding model: {status['vector_store_stats']['model_name']}

**Models:**
• Generation model: {status.get('model_info', {}).get('generation_model', 'N/A')}
• GPU available: {status.get('model_info', {}).get('gpu_available', False)}
"""
        return info_text

    # Create Gradio interface
    with gr.Blocks(
        title="RAG System - AI Research Papers QA",
        theme=gr.themes.Soft()
    ) as interface:

        gr.Markdown("""
        # 🤖 RAG System: AI Research Papers Q&A

        **Upload your AI research papers and ask questions about them!**

        This system uses Retrieval-Augmented Generation to:
        - 📖 Process and understand your research papers
        - 🔍 Find relevant information for your questions
        - 🤖 Generate accurate answers with source citations
        """)

        with gr.Tab("📤 Upload & Setup"):
            gr.Markdown("### Step 1: Upload Your PDF Research Papers")

            file_upload = gr.File(
                label="Select PDF Files",
                file_count="multiple",
                file_types=[".pdf"],
                height=100
            )

            setup_btn = gr.Button("🚀 Process Documents", variant="primary", size="lg")
            setup_output = gr.Textbox(
                label="Setup Status",
                lines=10,
                interactive=False,
                placeholder="Upload PDFs and click 'Process Documents' to begin..."
            )

            # Sample data button for demo
            gr.Markdown("---")
            gr.Markdown("### Or Try with Sample Data")
            sample_btn = gr.Button("🎯 Use Sample Data (Demo)", variant="secondary")

            def setup_sample():
                success = setup_with_sample_data()
                if success:
                    status = rag_system.get_system_status()
                    return f"""✅ Sample System Ready!

This is a demo with sample AI research paper content.
You can now ask questions like:
• "What is the main contribution of this paper?"
• "What methodology was used?"
• "What were the results?"

📊 System loaded with {status['documents_info']['total_chunks']} text chunks."""
                else:
                    return "❌ Failed to setup sample data."

            sample_btn.click(setup_sample, outputs=[setup_output])
            setup_btn.click(handle_file_upload, inputs=[file_upload], outputs=[setup_output])

        with gr.Tab("❓ Ask Questions"):
            gr.Markdown("### Ask Questions About Your Research Papers")

            with gr.Row():
                with gr.Column(scale=4):
                    question_input = gr.Textbox(
                        label="Your Question",
                        placeholder="e.g., What are the main contributions of this research?",
                        lines=3
                    )
                with gr.Column(scale=1):
                    ask_btn = gr.Button("🔍 Get Answer", variant="primary", size="lg")

            with gr.Row():
                with gr.Column():
                    answer_output = gr.Textbox(
                        label="Answer",
                        lines=8,
                        interactive=False
                    )
                with gr.Column():
                    sources_output = gr.Textbox(
                        label="Sources & Citations",
                        lines=8,
                        interactive=False
                    )

            # Sample questions
            gr.Markdown("### 💡 Sample Questions to Try:")
            sample_questions = [
                "What are the main contributions of this research?",
                "What methodology was used in this study?",
                "What are the key findings and results?",
                "What are the limitations mentioned?",
                "What future work is suggested?"
            ]

            for q in sample_questions:
                sample_q_btn = gr.Button(f"📝 {q}", variant="secondary", size="sm")
                sample_q_btn.click(lambda x=q: x, outputs=[question_input])

            ask_btn.click(
                handle_question,
                inputs=[question_input],
                outputs=[answer_output, sources_output]
            )

        with gr.Tab("ℹ️ System Info"):
            gr.Markdown("### System Status and Information")

            info_btn = gr.Button("🔄 Refresh System Info", variant="primary")
            info_output = gr.Textbox(
                label="System Information",
                lines=15,
                interactive=False
            )

            info_btn.click(show_system_info, outputs=[info_output])

            # Auto-load info on tab open
            interface.load(show_system_info, outputs=[info_output])

    return interface

# Create the interface
print("\n🎨 Creating Gradio Interface...")
interface = create_gradio_interface()
print("✅ Interface ready!")


🎨 Creating Gradio Interface...
✅ Interface ready!


**Interface Features:**

*   Document upload/status panel

*   Q&A with source citations

*   System diagnostics view

# **Cell 9: Runtime Controls**
**Launch commands:**

In [95]:
def run_system():
    """Launch the complete system"""
    print("🚀 Launching RAG System...")
    print("="*60)
    print("📋 Instructions:")
    print("1. Run this cell to launch the web interface")
    print("2. Upload your PDF research papers in the 'Upload & Setup' tab")
    print("3. Wait for processing to complete")
    print("4. Go to 'Ask Questions' tab and start asking!")
    print("="*60)

    # Launch interface
    interface.launch(
        share=True,  # Creates a public shareable link
        debug=True,
        height=800
    )

# Quick test function
def quick_test():
    """Quick test with sample data"""
    print("🧪 Quick Test Mode")
    print("-" * 30)

    # Setup with sample data
    if setup_with_sample_data():
        print("\n✅ Sample system ready!")

        # Test questions
        test_questions = [
            "What is this paper about?",
            "What methodology was used?",
            "What were the main results?"
        ]

        for question in test_questions:
            print(f"\n❓ Question: {question}")
            result = rag_system.ask_question(question, num_sources=2)
            print(f"✅ Answer: {result['answer'][:200]}...")
            print(f"📚 Sources: {len(result['sources'])} found")
            print(f"🎯 Confidence: {result['confidence']}")
    else:
        print("❌ Test setup failed")

**Two Usage Modes:**

*   Full web interface (run_system())

*   CLI-style testing (quick_test())

In [ ]:
run_system()

🚀 Launching RAG System...
📋 Instructions:
1. Run this cell to launch the web interface
2. Upload your PDF research papers in the 'Upload & Setup' tab
3. Wait for processing to complete
4. Go to 'Ask Questions' tab and start asking!
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://82180ce9a67366b9d2.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


🚀 Setting up RAG system...
Step 1: Loading PDFs...
📖 Loading: 1706.03762v7.pdf
✅ Loaded 15 pages from 1706.03762v7.pdf
📖 Loading: 2005.11401v4.pdf
✅ Loaded 19 pages from 2005.11401v4.pdf
📖 Loading: 2005.14165v4.pdf
✅ Loaded 75 pages from 2005.14165v4.pdf

Step 2: Creating text chunks...
🔪 Creating text chunks...
✅ Created 461 chunks

Step 3: Creating vector embeddings...
⚡ Creating vector embeddings...
📊 Processing 461 chunks...
✅ Vector store created successfully!
📈 Vector dimension: 384
📚 Total vectors: 461

🎉 RAG System Setup Complete!
📚 Loaded: 109 pages from 3 PDFs
🔪 Created: 461 text chunks
⚡ Vector store ready with 461 embeddings


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


🔍 Processing question: 'What are the main components of a RAG model, and how do they interact?'
🔍 Found 3 relevant chunks for query
🤖 Generating answer...
✅ Answer generated (confidence: 0.14800000190734863)
